In [117]:
import twint
import numpy as np
import pandas as pd

In [118]:
df = pd.read_csv("../data/raw/TweetsRaw.csv", error_bad_lines=False,lineterminator='\n')

/Users/avinaashpadman/opt/anaconda3/envs/cardanotwittersentiment/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version.





The twitter data was scrapped using a library called snscraper. The script I wrote for that is in the src file as scrapeTweets.py. The API allowed me to scrape 200000 tweets from 2021-08-31 to 2021-08-08 which I then saved in my raw data file as TweetsRaw. It scrapped tweets that contained the word "cardano" in it.  

In [119]:
df.tail()

,Unnamed: 0,Datetime,Tweet ID,Text,User
199996,199996,2021-08-08 22:29:41+00:00,1424498074865504257,@Fougias_ @elliotrades @cryptoknitties @AdaDolls @CardinosNFT @CardanoKidz @spacebudzNFT @CardanoBits @the_hoskinsons @CardanoMonsters @KingsCardano Nice one. I will look into these! What is the marketplace for cardano?,ConservationAR
199997,199997,2021-08-08 22:29:40+00:00,1424498068418826243,"How Cardano can help in Ethiopia, El Salvador and other developing countries https://t.co/rjAgSRQfB0",CryptoBerts
199998,199998,2021-08-08 22:29:39+00:00,1424498066137264136,"@eVIRALToken has been changing the game since inception and will continue to do so. Say goodbye to #rugpulls #cryptomarket, any project launched on the #eviral platform is a safe project. #ethereum #Bitcoin #cardano #Solana #BNB #BinanceSmartChain #Binance #Uniswap #PancakeSwap https://t.co/J3HUxKhfoG",kloflin86
199999,199999,2021-08-08 22:29:27+00:00,1424498014660481026,$ADA 24 Hours Stats #Cardano\n\n24h High: $1.49131\n24h Average: $1.45527\n24h Low: $1.40946,Altcoin_Track
200000,200000,2021-08-08 22:28:13+00:00,1424497705410342918,@project_takumi @cardano_bull @JonMunitz I’m talking about how they keep calling themselves progressives and now they refer it to the people who praise the squad,ForYang4


In [22]:
df.shape

(200001, 5)

In [23]:
df = df.drop_duplicates(subset = ["Text"])

In [24]:
df.shape

(195620, 5)

First thing that I decided to do was to make sure that any duplicate tweets are removed so that the data reflects the sentiment value of each unique tweet. This removes around 4500 duplicated tweets

In [25]:
import re
import nltk 
import string
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

First we have to process the tweets into words that can be more easily analyzed by the sentiment analyzer. I will use re to process the text. We want to remove any @username words and retweets. Using the re.sub function, we can substitute a certain pattern with a blank text, effectively removing the pattern from the text.

In [115]:
df.sample(10)

,Unnamed: 0,Datetime,Tweet ID,Text,User,sentimentScore
152232,152232,2021-08-15 01:44:06+00:00,1426721329051365379,@AdrianoFeria @TopherTerps @AstraStake @KaizenCrypto @AnthonyCardano @benohanlon @IOHK_Charles You don't think that the potential of thousands of project onboarding and building on Cardano is good fundamentals? Some people can't be helped...,_eLaPs_,1
24892,24892,2021-08-29 05:19:45+00:00,1431849026593251333,😩 I feel so poor looking at some of those #CNFT prices. 😳 #CardanoNFT #Cardano #ADA #CNFTs,JaY_RoXXI,-1
105912,105912,2021-08-20 19:15:48+00:00,1428797935400148995,#Cardano $ADA #ETH #BTC #BITCOIN\n\nStake and join our Discord for more Cardano https://t.co/XslAYeUIgh\n https://t.co/7k3PWqaakc,Everest_ADA,-1
192130,192130,2021-08-10 15:55:21+00:00,1425123611778732032,"@cardano_daily @liqwidfinance @MinswapDEX @nftmakerio @ErgoDex @QuestPool @InputOutputHK @emurgo_io @ergoplatformorg Thank you so much for building this! (And of coz, for the shoutout :D)",repsistance,1
2226,2226,2021-08-31 18:40:04+00:00,1432775209090641925,Un analyste fait des prévisions pessimistes pour Cardano – « il est temps de vendre » https://t.co/axxv1dqAtb,LaCryptoMonnai1,0
198529,198529,2021-08-09 09:55:41+00:00,1424670710295900164,"in next cicle! just hodl! 🚀🤑\n\n#Bitcoin to $350,000-450,000\n#Ethereum to $10,000-17,500\n#Binance to $3000-5000\n#Polkadot to $250-350\n#Chainlink to $250-350\n#Cardano to $10-20\n#Zilliqa to $5-7\n#Elrond to $500-750\n#DIA to $50-75",c_wallet_vip,0
173501,173501,2021-08-13 04:31:55+00:00,1426038783699345408,@SIPO_Tokyo @lovelacepool1 @Cardano_Afrik @ADABreathes @CardanoArmy @dolcapool @TheDUCKpool1 @Dow_shi1 @Gr88nCardano @beestaking @kurokuma_hunny @leon_stake_pool ありがとうございます🚀,deisensei,0
197485,197485,2021-08-09 15:00:59+00:00,1424747541631934468,#Cardano #ADA CatalystFUND5資金供給された70プロジェクト＋5つのコミュニティチャレンジFUND7に設定されました🔥その日本語概要一覧を作成しました。\n\n資金供給決定されたプロジェクトおめでとうございます！！🔥🔥🔥🔥🔥🔥🔥🔥🔥\n\nhttps://t.co/FKKUUWHLyW https://t.co/slVj5KFB8U,yuta_cryptox,0
146362,146362,2021-08-15 21:04:23+00:00,1427013323921448960,"@ellipalwallet @Cardano What problem that happens with ADA that when trying to withdraw my funds it does not allow me to tell me an invalid address, to an address to which I had previously sent ADA?",IgmelV,-1
6374,6374,2021-08-31 10:22:55+00:00,1432650099855794187,@SebastienGllmt U need permission to build on cardano,Cr03Sinister,0


In [116]:
removeRT = re.sub( 'RT \@'  )

TypeError: sub() missing 2 required positional arguments: 'repl' and 'string'

In [26]:
positive = 0
negative = 0
neutral= 0
polarity = 0
SentimentScore = []


In [33]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/avinaashpadman/nltk_data...


True

In [34]:
for idx, tweet in df.iterrows():
    #print(tweet.Text)
    analysis = TextBlob(tweet.Text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.Text)
    neg = score['neg']
    pos = score['pos']
    neu = score['neg']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        SentimentScore.append(-1)
    
    if pos > neg:
        SentimentScore.append(1)
    
    if pos == neg:
        SentimentScore.append(0)
        
        

In [40]:
df.head()

,Unnamed: 0,Datetime,Tweet ID,Text,User,sentimentScore
0,0,2021-08-31 23:59:57+00:00,1432855712657412096,@sereingives @nft_sol @nfts_cardano Done,yedamdashing1,0
1,1,2021-08-31 23:59:49+00:00,1432855676729106436,"@jeremeybivens This is a way to store your keys for any type of storage. Yea, you can stake from hot and cold storage on Cardano!",TheADAApe,0
2,2,2021-08-31 23:59:47+00:00,1432855671343656963,What’s great about NFT’s on Cardano? They’re fun! #cryptodino @cryptodino_io #NFTCommunity #NFT #ADA #ADAgang #cardano #CardanoNFT https://t.co/YdiXy7AZrl,LarryMiles,1
3,3,2021-08-31 23:59:35+00:00,1432855621192364032,Hoskinson: These are the circumstances in which Cardano $ADA and Solana $SOL will collaborate,CyrilXbt,0
4,4,2021-08-31 23:58:53+00:00,1432855443617902595,@Cardano_Robin @psychADAlias @Champagnemami02 @tr3090 @w909924,j20702050,0


In [56]:
df["sentimentScore"].value_counts()

 0    95176
 1    76818
-1    23626
Name: sentimentScore, dtype: int64

In [39]:
df["Datetime"] = pd.to_datetime(df["Datetime"])

In [37]:
df.dtypes

Unnamed: 0         int64
Datetime          object
Tweet ID           int64
Text              object
User              object
sentimentScore     int64
dtype: object

In [63]:
df.groupby([pd.Grouper(key='Datetime', freq='D'),"sentimentScore"]).count()

Unnamed: 0  Tweet ID  Text  User
Datetime                  sentimentScore                                  
2021-08-08 00:00:00+00:00 -1                      26        26    26    26
                           0                     112       112   112   112
                           1                      95        95    95    95
2021-08-09 00:00:00+00:00 -1                     431       431   431   431
                           0                    1979      1979  1979  1979
...                                              ...       ...   ...   ...
2021-08-30 00:00:00+00:00  0                    4147      4147  4147  4147
                           1                    3554      3554  3554  3554
2021-08-31 00:00:00+00:00 -1                    1133      1133  1133  1133
                           0                    4242      4242  4242  4242
                           1                    3525      3525  3525  3525

[72 rows x 4 columns]

In [82]:
 DailyTweetSentiment =  df.groupby(pd.Grouper(key='Datetime', freq='D'))["sentimentScore"].sum()

In [83]:
 DailyTweetSentiment = pd.DataFrame(DailyTweetSentiment)

In [113]:
DailyTweetSentiment.head()

,sentimentScore
Datetime,
2021-08-08 00:00:00+00:00,69
2021-08-09 00:00:00+00:00,1268
2021-08-10 00:00:00+00:00,1681
2021-08-11 00:00:00+00:00,2373
2021-08-12 00:00:00+00:00,1710


The DailyTweetSentiment dataframe contains the cumulative sentimentScore for each day since the 8th of August to the 31st of August

In [84]:
import plotly.express as px

In [89]:
px.bar(DailyTweetSentiment)

In [99]:
DailyTweetSentimentByScore = df.groupby([pd.Grouper(key='Datetime', freq='D'),"sentimentScore"])["User"].count()

In [101]:
DailyTweetSentimentByScore.head()

,sentimentScore,User
Datetime,,
2021-08-08 00:00:00+00:00,-1,26
2021-08-08 00:00:00+00:00,0,112
2021-08-08 00:00:00+00:00,1,95
2021-08-09 00:00:00+00:00,-1,431
2021-08-09 00:00:00+00:00,0,1979


In [100]:
DailyTweetSentimentByScore = DailyTweetSentimentByScore.reset_index(level = ["sentimentScore"])

In [105]:
px.bar(DailyTweetSentimentByScore, color = "sentimentScore")

In [111]:
PosSentDailyTweets = DailyTweetSentimentByScore[DailyTweetSentimentByScore["sentimentScore"] == 1]
 PosSentDailyTweets = PosSentDailyTweets.drop("sentimentScore", axis = 1)

IndentationError: unexpected indent (3794561307.py, line 2)

In [112]:
px.bar(PosSentDailyTweets)